login to gugging face

from huggingface_hub import notebook_login
notebook_login()

In [2]:
from datasets import Dataset
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import ast
import json

In [153]:
df = pd.read_csv("/Users/software/Desktop/Hieroglyphics_nlp/code/language_model/Translations.csv",sep=";")

In [154]:
def convert_to_dict(string):
    try:
        string = string.replace("'", '"')
        return json.loads(string)
    except json.JSONDecodeError as e:
        
        return None
df.drop("Index",axis=1)
df['Translations'] = df['Translations'].apply(convert_to_dict)
df.to_csv("hello.csv")

In [155]:
df.drop("Index",axis=1)
df["Translations"]

0           {'Sequence': 'A1', 'Definition': 'I, me, my '}
1        {'Sequence': 'A1 A1 A1', 'Definition': 'man, m...
2               {'Sequence': 'A2', 'Definition': 'drink '}
3        {'Sequence': 'A6', 'Definition': 'pure, purify...
4        {'Sequence': 'A6A', 'Definition': 'pure, purif...
                               ...                        
11718    {'Sequence': 'Aa32 X1 G4 T14 A1 N33A', 'Defini...
11719    {'Sequence': 'Aa32 X1 N25', 'Definition': 'Nub...
11720    {'Sequence': 'Aa32 X1 N25 T14 A1 N33A', 'Defin...
11721    {'Sequence': 'Aa32 X1 Z4 N33 Z2', 'Definition'...
11722    {'Sequence': 'Aa32 X1 Z8', 'Definition': 'Nubi...
Name: Translations, Length: 11723, dtype: object

In [156]:
# lets get our custom dataset in order

# Replace 'your_dataset.csv' with the actual file path of your CSV file
file_path = 'hieroglyph_translation_dictionary_storage_final.csv'

# Load your dataset into a Dataset object
hieroglyph_translation = Dataset.from_pandas(df)


# Print a sample from your dataset
print(hieroglyph_translation)



Dataset({
    features: ['Index', 'Translations'],
    num_rows: 11723
})


In [157]:
train_ds = Dataset.from_pandas(df, split="train")
test_ds = Dataset.from_pandas(df, split="test")

In [158]:
import json
type(hieroglyph_translation["Translations"][1])



dict

In [159]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [173]:
source_lang = "Sequence"
target_lang = "Definition"
prefix = "Translate hieroglyph unicode sequence to English: "

def preprocess_function(examples):
    #try:
    inputs=[]
    targets=[]
    x=0
    for example in examples["Translations"]:
        try:
            inputs.append(prefix+example[source_lang])
            targets.append(example[target_lang])

        except:
            print(x)
            x=x+1

#     inputs=[prefix + example["Sequence"] for example in examples["Translations"]]
#     targets = [example["Definition"] for example in examples["Translations"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=1000, truncation=True)
    return model_inputs

    #except:
    #    print("Aiyayo Vada Poche")
    
tokenized_hieroglyph_translations = train_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/11723 [00:00<?, ? examples/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


ArrowInvalid: Column 2 named input_ids expected length 1000 but got length 985

In [163]:
tokenized_hieroglyph_translations[0]

{'Index': 0, 'Translations': {'Definition': 'I, me, my ', 'Sequence': 'A1'}}

In [114]:
tokenized_hieroglyph_translations[2]

{'Index': 2, 'Translations': {'Definition': 'drink ', 'Sequence': 'A2'}}

Example 1:


KeyError: 'input_ids'

In [147]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [148]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [149]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [170]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/AnushS/my_awesome_opus_books_model into local empty directory.
/Users/software/anaconda3/envs/mlp/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


IndexError: Invalid key: 10979 is out of bounds for size 0

In [151]:
trainer.push_to_hub()

NameError: name 'trainer' is not defined